In [9]:
#task1
import pandas as pd
import sqlite3


df = pd.read_csv('dataset_transactions.csv')


conn = sqlite3.connect(':memory:')
df.to_sql('transactions', conn, index=False, if_exists='replace')


query = """
    SELECT 
        City,
        SUM(Amount) as Total_Spend,
        (SUM(Amount) * 100.0 / (SELECT SUM(Amount) FROM transactions)) as Percentage_Contribution
    FROM 
        transactions
    GROUP BY 
        City
    ORDER BY 
        Total_Spend DESC
    LIMIT 5;
"""


result_df = pd.read_sql_query(query, conn)


result_df.to_csv('output_task1.csv', index=False)


conn.close()


In [14]:
#task2
import pandas as pd
import sqlite3


df = pd.read_csv('dataset_transactions.csv')


df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y')


conn = sqlite3.connect(':memory:')
df.to_sql('transactions', conn, index=False, if_exists='replace')


query = """
    WITH Monthly_Spends AS (
        SELECT 
            "Card Type" as Card_Type,
            strftime('%Y-%m', Date) as Month,
            SUM(Amount) as Total_Spend
        FROM 
            transactions
        GROUP BY 
            Card_Type, Month
    ),
    Max_Spends AS (
        SELECT 
            Card_Type,
            MAX(Total_Spend) as Max_Spend
        FROM 
            Monthly_Spends
        GROUP BY 
            Card_Type
    )
    SELECT 
        m.Card_Type,
        m.Month,
        m.Total_Spend
    FROM 
        Monthly_Spends m
    JOIN 
        Max_Spends ms
    ON 
        m.Card_Type = ms.Card_Type AND
        m.Total_Spend = ms.Max_Spend
"""
    

result_df = pd.read_sql_query(query, conn)
result_df.to_csv('output_task2.csv', index=False)


conn.close()


In [15]:
#task3
import pandas as pd
import sqlite3

df = pd.read_csv('dataset_transactions.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y')
conn = sqlite3.connect(':memory:')
df.to_sql('transactions', conn, index=False, if_exists='replace')

query = """
    SELECT 
        *
    FROM (
        SELECT 
            *,
            SUM(Amount) OVER (PARTITION BY "Card Type" ORDER BY Date) as Cumulative_Spend
        FROM 
            transactions
    ) 
    WHERE 
        Cumulative_Spend >= 1000000
    GROUP BY 
        "Card Type"
    HAVING 
        MIN(Date)
"""

result_df = pd.read_sql_query(query, conn)
result_df.to_csv('output_task3.csv', index=False)
conn.close()


In [16]:
#task4
import pandas as pd
import sqlite3

df = pd.read_csv('dataset_transactions.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y')
conn = sqlite3.connect(':memory:')
df.to_sql('transactions', conn, index=False, if_exists='replace')

query = """
    SELECT 
        City,
        (SUM(Amount) * 100.0 / (SELECT SUM(Amount) FROM transactions WHERE "Card Type" = 'Gold')) AS Percentage_Spend
    FROM 
        transactions
    WHERE 
        "Card Type" = 'Gold'
    GROUP BY 
        City
    ORDER BY 
        Percentage_Spend ASC
    LIMIT 1
"""

result_df = pd.read_sql_query(query, conn)
result_df.to_csv('output_task4.csv', index=False)
conn.close()


In [17]:
#task5
import pandas as pd
import sqlite3

df = pd.read_csv('dataset_transactions.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y')
conn = sqlite3.connect(':memory:')
df.to_sql('transactions', conn, index=False, if_exists='replace')

query = """
    WITH ExpenseTotals AS (
        SELECT 
            City,
            "Exp Type",
            SUM(Amount) AS Total_Spend
        FROM 
            transactions
        GROUP BY 
            City, "Exp Type"
    ),
    RankedExpenses AS (
        SELECT 
            City,
            "Exp Type",
            Total_Spend,
            RANK() OVER (PARTITION BY City ORDER BY Total_Spend DESC) AS SpendRankDesc,
            RANK() OVER (PARTITION BY City ORDER BY Total_Spend ASC) AS SpendRankAsc
        FROM 
            ExpenseTotals
    )
    SELECT 
        City,
        MAX(CASE WHEN SpendRankDesc = 1 THEN "Exp Type" END) AS highest_expense_type,
        MAX(CASE WHEN SpendRankAsc = 1 THEN "Exp Type" END) AS lowest_expense_type
    FROM 
        RankedExpenses
    GROUP BY 
        City
"""

result_df = pd.read_sql_query(query, conn)
result_df.to_csv('output_task5.csv', index=False)
conn.close()


In [18]:
#task6
import pandas as pd
import sqlite3

df = pd.read_csv('dataset_transactions.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y')
conn = sqlite3.connect(':memory:')
df.to_sql('transactions', conn, index=False, if_exists='replace')

query = """
    SELECT 
        "Exp Type" AS expense_type,
        SUM(CASE WHEN Gender = 'F' THEN Amount ELSE 0 END) * 100.0 / SUM(Amount) AS female_percentage_contribution
    FROM 
        transactions
    GROUP BY 
        expense_type
"""

result_df = pd.read_sql_query(query, conn)
result_df.to_csv('output_task6.csv', index=False)
conn.close()


In [20]:
#task7
import pandas as pd
import sqlite3

df = pd.read_csv('dataset_transactions.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y')

conn = sqlite3.connect(':memory:')
df.to_sql('transactions', conn, index=False, if_exists='replace')

query = """
WITH MonthlySpend AS (
    SELECT 
        strftime('%Y-%m', Date) AS month,
        [Card Type],
        [Exp Type],
        SUM(Amount) AS total_amount
    FROM transactions
    GROUP BY month, [Card Type], [Exp Type]
),
Growth AS (
    SELECT 
        current.[Card Type],
        current.[Exp Type],
        (current.total_amount - COALESCE(previous.total_amount, 0)) AS growth
    FROM MonthlySpend AS current
    LEFT JOIN MonthlySpend AS previous
    ON current.[Card Type] = previous.[Card Type]
    AND current.[Exp Type] = previous.[Exp Type]
    AND strftime('%Y-%m', current.month) = strftime('%Y-%m', previous.month, '+1 month')
    WHERE current.month = '2014-01'
)
SELECT 
    [Card Type],
    [Exp Type],
    growth
FROM Growth
ORDER BY growth DESC
LIMIT 1;
"""

result = pd.read_sql_query(query, conn)
result.to_csv('output_task7.csv', index=False)

conn.close()


In [21]:
#task8
import pandas as pd
import sqlite3

df = pd.read_csv('dataset_transactions.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y')

conn = sqlite3.connect(':memory:')
df.to_sql('transactions', conn, index=False, if_exists='replace')

query = """
WITH WeekendTransactions AS (
    SELECT 
        City,
        Amount,
        strftime('%w', Date) AS weekday
    FROM transactions
    WHERE strftime('%w', Date) IN ('0', '6')  -- 0 = Sunday, 6 = Saturday
),
CitySpend AS (
    SELECT 
        City,
        SUM(Amount) AS total_spend,
        COUNT(*) AS total_transactions
    FROM WeekendTransactions
    GROUP BY City
)
SELECT 
    City,
    total_spend / total_transactions AS spend_to_transaction_ratio
FROM CitySpend
ORDER BY spend_to_transaction_ratio DESC
LIMIT 1;
"""

result = pd.read_sql_query(query, conn)
result.to_csv('output_task8.csv', index=False)

conn.close()


In [22]:
#task9
import pandas as pd
import sqlite3

df = pd.read_csv('dataset_transactions.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y')

conn = sqlite3.connect(':memory:')
df.to_sql('transactions', conn, index=False, if_exists='replace')

query = """
WITH RankedTransactions AS (
    SELECT 
        City,
        Date,
        ROW_NUMBER() OVER (PARTITION BY City ORDER BY Date) AS transaction_rank
    FROM transactions
),
TransactionStats AS (
    SELECT 
        City,
        MIN(Date) AS first_transaction_date,
        MAX(CASE WHEN transaction_rank = 500 THEN Date END) AS date_500th_transaction
    FROM RankedTransactions
    GROUP BY City
    HAVING COUNT(*) >= 500
),
DaysTo500thTransaction AS (
    SELECT
        City,
        JULIANDAY(date_500th_transaction) - JULIANDAY(first_transaction_date) AS days_to_500
    FROM TransactionStats
)
SELECT 
    City,
    days_to_500
FROM DaysTo500thTransaction
ORDER BY days_to_500 ASC
LIMIT 1;
"""

result = pd.read_sql_query(query, conn)
result.to_csv('output_task9.csv', index=False)

conn.close()
